## Set up 

In [ ]:
#pip install google-cloud-bigquery

In [21]:
from google.cloud import bigquery
import os
import pandas as pd
import datetime as dt
import numpy as np
from pandas.io import gbq

In [22]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
%cd "..." # insert your path here in Google Colab. Skip this one if using local 

/content/drive/Shareddrives/Tieu_Quyen/Data_Science/Data Engineering/DE2


## Data collection API

In [24]:
import requests
import json
from requests.auth import HTTPBasicAuth
import pandas as pd
from pandas.io.json import json_normalize

In [25]:
#: API key
keyVal='xxx' #input your API key here
auth = HTTPBasicAuth(keyVal, '')

In [26]:
# Trending media_type
api_endpoint=f'https://api.themoviedb.org/3/discover/movie?api_key={keyVal}&language=en-US&sort_by=popularity.desc&include_adult=true&include_video=true&page=1&with_watch_monetization_types=flatrate'
data=requests.get(api_endpoint, auth=auth).json()

In [27]:
# Find out total number of pages
api_endpoint=f'https://api.themoviedb.org/3/discover/movie?api_key={keyVal}&language=en-US&sort_by=popularity.desc&include_adult=true&include_video=true&page=1&with_watch_monetization_types=flatrate'
data = requests.get(url=api_endpoint,auth=auth).json()
total_pages = data['total_pages']
total_pages

500

In [28]:
data = pd.DataFrame([])
for page in range (1, total_pages):
    api_endpoint=f'https://api.themoviedb.org/3/discover/movie?api_key={keyVal}&language=en-US&sort_by=popularity.desc&include_adult=true&include_video=true&page={page}&with_watch_monetization_types=flatrate'
    df=requests.get(api_endpoint, auth=auth).json()
    df_new = pd.json_normalize(df['results'])
    data = pd.concat([data, df_new])
    page += 1

In [29]:
data.count()

adult                9980
backdrop_path        9577
genre_ids            9980
id                   9980
original_language    9980
original_title       9980
overview             9980
popularity           9980
poster_path          9939
release_date         9969
title                9980
video                9980
vote_average         9980
vote_count           9980
dtype: int64

## Data Pre-processing
It is because of big data that student's account can not handle, I just filtered to make it smaller size for next steps 

In [30]:
movie = data[['adult','id','original_language','overview','popularity','release_date','title','vote_average','vote_count']]
movie.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9980 entries, 0 to 19
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   adult              9980 non-null   bool   
 1   id                 9980 non-null   int64  
 2   original_language  9980 non-null   object 
 3   overview           9980 non-null   object 
 4   popularity         9980 non-null   float64
 5   release_date       9969 non-null   object 
 6   title              9980 non-null   object 
 7   vote_average       9980 non-null   float64
 8   vote_count         9980 non-null   int64  
dtypes: bool(1), float64(2), int64(2), object(4)
memory usage: 711.5+ KB


In [31]:
movie['year'] = pd.to_datetime(movie['release_date']).dt.year
movie.head()


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,adult,id,original_language,overview,popularity,release_date,title,vote_average,vote_count,year
0,False,580489,en,After finding a host body in investigative rep...,7235.697,2021-09-30,Venom: Let There Be Carnage,7.2,4451,2021.0
1,False,512195,en,An Interpol-issued Red Notice is a global aler...,4394.992,2021-11-04,Red Notice,6.8,1997,2021.0
2,False,617653,en,King Charles VI declares that Knight Jean de C...,2478.809,2021-10-13,The Last Duel,7.6,904,2021.0
3,False,566525,en,Shang-Chi must confront the past he thought he...,2639.797,2021-09-01,Shang-Chi and the Legend of the Ten Rings,7.8,4226,2021.0
4,False,568124,en,"The tale of an extraordinary family, the Madri...",2639.096,2021-11-24,Encanto,7.4,316,2021.0


In [32]:
movie['month'] = pd.to_datetime(movie['release_date']).dt.month
movie.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,adult,id,original_language,overview,popularity,release_date,title,vote_average,vote_count,year,month
0,False,580489,en,After finding a host body in investigative rep...,7235.697,2021-09-30,Venom: Let There Be Carnage,7.2,4451,2021.0,9.0
1,False,512195,en,An Interpol-issued Red Notice is a global aler...,4394.992,2021-11-04,Red Notice,6.8,1997,2021.0,11.0
2,False,617653,en,King Charles VI declares that Knight Jean de C...,2478.809,2021-10-13,The Last Duel,7.6,904,2021.0,10.0
3,False,566525,en,Shang-Chi must confront the past he thought he...,2639.797,2021-09-01,Shang-Chi and the Legend of the Ten Rings,7.8,4226,2021.0,9.0
4,False,568124,en,"The tale of an extraordinary family, the Madri...",2639.096,2021-11-24,Encanto,7.4,316,2021.0,11.0


In [33]:
movie_list = movie.where((movie['original_language']=='en') & (movie['year']==2021) & (movie['vote_count']>0))
movie_list.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9980 entries, 0 to 19
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   adult              515 non-null    float64
 1   id                 515 non-null    float64
 2   original_language  515 non-null    object 
 3   overview           515 non-null    object 
 4   popularity         515 non-null    float64
 5   release_date       515 non-null    object 
 6   title              515 non-null    object 
 7   vote_average       515 non-null    float64
 8   vote_count         515 non-null    float64
 9   year               515 non-null    float64
 10  month              515 non-null    float64
dtypes: float64(7), object(4)
memory usage: 935.6+ KB


In [34]:
movie_list=movie_list.dropna(how='any') 
movie_list.head()

,adult,id,original_language,overview,popularity,release_date,title,vote_average,vote_count,year,month
0,0.0,580489.0,en,After finding a host body in investigative rep...,7235.697,2021-09-30,Venom: Let There Be Carnage,7.2,4451.0,2021.0,9.0
1,0.0,512195.0,en,An Interpol-issued Red Notice is a global aler...,4394.992,2021-11-04,Red Notice,6.8,1997.0,2021.0,11.0
2,0.0,617653.0,en,King Charles VI declares that Knight Jean de C...,2478.809,2021-10-13,The Last Duel,7.6,904.0,2021.0,10.0
3,0.0,566525.0,en,Shang-Chi must confront the past he thought he...,2639.797,2021-09-01,Shang-Chi and the Legend of the Ten Rings,7.8,4226.0,2021.0,9.0
4,0.0,568124.0,en,"The tale of an extraordinary family, the Madri...",2639.096,2021-11-24,Encanto,7.4,316.0,2021.0,11.0


In [35]:
movie_list=movie_list.drop_duplicates()
movie_list.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 515 entries, 0 to 5
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   adult              515 non-null    float64
 1   id                 515 non-null    float64
 2   original_language  515 non-null    object 
 3   overview           515 non-null    object 
 4   popularity         515 non-null    float64
 5   release_date       515 non-null    object 
 6   title              515 non-null    object 
 7   vote_average       515 non-null    float64
 8   vote_count         515 non-null    float64
 9   year               515 non-null    float64
 10  month              515 non-null    float64
dtypes: float64(7), object(4)
memory usage: 48.3+ KB


In [36]:
final_movie = movie_list[['adult','id','overview','popularity','release_date','title','vote_average','vote_count']]
final_movie.head()

,adult,id,overview,popularity,release_date,title,vote_average,vote_count
0,0.0,580489.0,After finding a host body in investigative rep...,7235.697,2021-09-30,Venom: Let There Be Carnage,7.2,4451.0
1,0.0,512195.0,An Interpol-issued Red Notice is a global aler...,4394.992,2021-11-04,Red Notice,6.8,1997.0
2,0.0,617653.0,King Charles VI declares that Knight Jean de C...,2478.809,2021-10-13,The Last Duel,7.6,904.0
3,0.0,566525.0,Shang-Chi must confront the past he thought he...,2639.797,2021-09-01,Shang-Chi and the Legend of the Ten Rings,7.8,4226.0
4,0.0,568124.0,"The tale of an extraordinary family, the Madri...",2639.096,2021-11-24,Encanto,7.4,316.0


In [37]:
final_movie.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 515 entries, 0 to 5
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   adult         515 non-null    float64
 1   id            515 non-null    float64
 2   overview      515 non-null    object 
 3   popularity    515 non-null    float64
 4   release_date  515 non-null    object 
 5   title         515 non-null    object 
 6   vote_average  515 non-null    float64
 7   vote_count    515 non-null    float64
dtypes: float64(5), object(3)
memory usage: 36.2+ KB


## Load data to GCP
This step is to load data directly to Google Cloud Platform (Database)

In [38]:
final_movie.to_gbq(destination_table='movie_dataset.table_name', # input your dataset.table_name here
                   project_id='xxx', # input your project id here
                   if_exists='replace')

1it [00:03,  3.34s/it]
